# วันที่ 3: การสร้าง Dashboard และวิเคราะห์กรณีศึกษา
## ช่วงบ่าย (13.00 น.–16.00 น.): กรณีศึกษา – วิเคราะห์ข้อมูลจากสถานการณ์จริง

## 2. การเตรียมข้อมูลสำหรับการวิเคราะห์

ในกรณีศึกษานี้ เราจะใช้ Python เพื่อเตรียมข้อมูลก่อนนำเข้า Power BI โดยจะเน้นการสร้างชุดข้อมูลตัวอย่างและการแปลงข้อมูลเพื่อให้เหมาะกับการวิเคราะห์

In [ ]:
# นำเข้าไลบรารีที่จำเป็น
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import random
import os

# กำหนดให้แสดงกราฟในโน้ตบุ๊ก
%matplotlib inline
# กำหนดสไตล์ของกราฟ
plt.style.use('seaborn-v0_8')
# กำหนดให้แสดงคอลัมน์ทั้งหมดในดาต้าเฟรม
pd.set_option('display.max_columns', None)

### 2.1 สร้างข้อมูลตัวอย่าง

เราจะสร้างข้อมูลตัวอย่างทั้ง 4 ชุดสำหรับใช้ในกรณีศึกษา

In [ ]:
# สร้างข้อมูลสินค้า
def create_product_data():
    """
    สร้างข้อมูลสินค้าตัวอย่างสำหรับการวิเคราะห์
    
    Returns:
        DataFrame ของข้อมูลสินค้า
    """
    categories = ['อิเล็กทรอนิกส์', 'เครื่องใช้ในบ้าน', 'เครื่องแต่งกาย', 'อาหารและเครื่องดื่ม', 'เครื่องสำอาง']
    subcategories = {
        'อิเล็กทรอนิกส์': ['สมาร์ทโฟน', 'แล็ปท็อป', 'แท็บเล็ต', 'ทีวี', 'กล้องถ่ายรูป'],
        'เครื่องใช้ในบ้าน': ['เครื่องซักผ้า', 'ตู้เย็น', 'เครื่องปรับอากาศ', 'เครื่องดูดฝุ่น', 'หม้อหุงข้าว'],
        'เครื่องแต่งกาย': ['เสื้อยืด', 'กางเกง', 'รองเท้า', 'กระเป๋า', 'เครื่องประดับ'],
        'อาหารและเครื่องดื่ม': ['เครื่องดื่ม', 'ขนมขบเคี้ยว', 'อาหารแห้ง', 'อาหารแช่แข็ง', 'ผลไม้'],
        'เครื่องสำอาง': ['ผลิตภัณฑ์ดูแลผิวหน้า', 'ผลิตภัณฑ์ดูแลผิวกาย', 'เครื่องสำอาง', 'น้ำหอม', 'ผลิตภัณฑ์ดูแลเส้นผม']
    }
    
    product_ids = []
    product_names = []
    product_categories = []
    product_subcategories = []
    product_prices = []
    product_costs = []
    
    product_id = 1000
    
    for category in categories:
        for subcategory in subcategories[category]:
            for i in range(5):  # สร้างสินค้า 5 รายการต่อหมวดหมู่ย่อย
                product_id += 1
                product_ids.append(product_id)
                product_name = f"{subcategory} รุ่น {i+1}"
                product_names.append(product_name)
                product_categories.append(category)
                product_subcategories.append(subcategory)
                
                # กำหนดราคาตามหมวดหมู่
                if category == 'อิเล็กทรอนิกส์':
                    price = random.randint(5000, 50000)
                elif category == 'เครื่องใช้ในบ้าน':
                    price = random.randint(2000, 30000)
                elif category == 'เครื่องแต่งกาย':
                    price = random.randint(500, 5000)
                elif category == 'อาหารและเครื่องดื่ม':
                    price = random.randint(20, 500)
                else:  # เครื่องสำอาง
                    price = random.randint(200, 3000)
                
                product_prices.append(price)
                # ต้นทุนประมาณ 60-80% ของราคาขาย
                cost = int(price * (random.randint(60, 80) / 100))
                product_costs.append(cost)
    
    products_df = pd.DataFrame({
        'product_id': product_ids,
        'product_name': product_names,
        'category': product_categories,
        'subcategory': product_subcategories,
        'price': product_prices,
        'cost': product_costs
    })
    
    return products_df

In [ ]:
# สร้างข้อมูลลูกค้า
def create_customer_data():
    """
    สร้างข้อมูลลูกค้าตัวอย่างสำหรับการวิเคราะห์
    
    Returns:
        DataFrame ของข้อมูลลูกค้า
    """
    regions = ['กรุงเทพและปริมณฑล', 'ภาคเหนือ', 'ภาคตะวันออกเฉียงเหนือ', 'ภาคกลาง', 'ภาคตะวันออก', 'ภาคตะวันตก', 'ภาคใต้']
    provinces = {
        'กรุงเทพและปริมณฑล': ['กรุงเทพมหานคร', 'นนทบุรี', 'ปทุมธานี', 'สมุทรปราการ', 'สมุทรสาคร', 'นครปฐม'],
        'ภาคเหนือ': ['เชียงใหม่', 'เชียงราย', 'ลำปาง', 'พิษณุโลก', 'นครสวรรค์'],
        'ภาคตะวันออกเฉียงเหนือ': ['ขอนแก่น', 'นครราชสีมา', 'อุดรธานี', 'อุบลราชธานี', 'สกลนคร'],
        'ภาคกลาง': ['พระนครศรีอยุธยา', 'ลพบุรี', 'สระบุรี', 'สุพรรณบุรี', 'สิงห์บุรี'],
        'ภาคตะวันออก': ['ชลบุรี', 'ระยอง', 'จันทบุรี', 'ตราด', 'ฉะเชิงเทรา'],
        'ภาคตะวันตก': ['กาญจนบุรี', 'ราชบุรี', 'เพชรบุรี', 'ประจวบคีรีขันธ์', 'ตาก'],
        'ภาคใต้': ['สงขลา', 'สุราษฎร์ธานี', 'นครศรีธรรมราช', 'ภูเก็ต', 'กระบี่']
    }
    
    genders = ['ชาย', 'หญิง']
    age_groups = ['18-24', '25-34', '35-44', '45-54', '55+']
    
    customer_ids = []
    customer_names = []
    customer_regions = []
    customer_provinces = []
    customer_genders = []
    customer_age_groups = []
    customer_registration_dates = []
    
    first_names_male = ['สมชาย', 'วิชัย', 'อนุชา', 'ธนากร', 'เอกราช', 'พงศธร', 'ทศพล', 'ปรีชา', 'อภิชาติ', 'ณัฐพงษ์']
    first_names_female = ['สมหญิง', 'วันดี', 'นภา', 'พิมพ์', 'วรรณา', 'อรุณี', 'สุนิสา', 'กัลยา', 'อัญชลี', 'นุชนาถ']
    last_names = ['สมบูรณ์', 'ใจดี', 'ทองคำ', 'รักเรียน', 'พงษ์พัฒน์', 'ศรีสมบูรณ์', 'บุญมา', 'พงษ์ศักดิ์', 'วงศ์สวัสดิ์', 'สุขสมบูรณ์']
    
    # กำหนดจำนวนลูกค้าต่อภูมิภาค
    customers_per_region = 50
    customer_id = 10000
    
    # สร้างวันที่ลงทะเบียนในช่วง 3 ปีที่ผ่านมา
    end_date = datetime.now()
    start_date = end_date - timedelta(days=3*365)  # 3 ปีย้อนหลัง
    
    for region in regions:
        for _ in range(customers_per_region):
            customer_id += 1
            customer_ids.append(customer_id)
            
            gender = random.choice(genders)
            if gender == 'ชาย':
                first_name = random.choice(first_names_male)
            else:
                first_name = random.choice(first_names_female)
                
            last_name = random.choice(last_names)
            customer_name = f"{first_name} {last_name}"
            customer_names.append(customer_name)
            
            customer_regions.append(region)
            province = random.choice(provinces[region])
            customer_provinces.append(province)
            
            customer_genders.append(gender)
            age_group = random.choice(age_groups)
            customer_age_groups.append(age_group)
            
            # วันที่ลงทะเบียนสุ่มในช่วง 3 ปีที่ผ่านมา
            days_delta = (end_date - start_date).days
            random_days = random.randint(0, days_delta)
            registration_date = start_date + timedelta(days=random_days)
            customer_registration_dates.append(registration_date.strftime('%Y-%m-%d'))
    
    customers_df = pd.DataFrame({
        'customer_id': customer_ids,
        'customer_name': customer_names,
        'region': customer_regions,
        'province': customer_provinces,
        'gender': customer_genders,
        'age_group': customer_age_groups,
        'registration_date': customer_registration_dates
    })
    
    return customers_df

In [ ]:
# สร้างข้อมูลร้านค้า
def create_store_data():
    """
    สร้างข้อมูลร้านค้าตัวอย่างสำหรับการวิเคราะห์
    
    Returns:
        DataFrame ของข้อมูลร้านค้า
    """
    regions = ['กรุงเทพและปริมณฑล', 'ภาคเหนือ', 'ภาคตะวันออกเฉียงเหนือ', 'ภาคกลาง', 'ภาคตะวันออก', 'ภาคตะวันตก', 'ภาคใต้']
    provinces = {
        'กรุงเทพและปริมณฑล': ['กรุงเทพมหานคร', 'นนทบุรี', 'ปทุมธานี', 'สมุทรปราการ', 'สมุทรสาคร', 'นครปฐม'],
        'ภาคเหนือ': ['เชียงใหม่', 'เชียงราย', 'ลำปาง', 'พิษณุโลก', 'นครสวรรค์'],
        'ภาคตะวันออกเฉียงเหนือ': ['ขอนแก่น', 'นครราชสีมา', 'อุดรธานี', 'อุบลราชธานี', 'สกลนคร'],
        'ภาคกลาง': ['พระนครศรีอยุธยา', 'ลพบุรี', 'สระบุรี', 'สุพรรณบุรี', 'สิงห์บุรี'],
        'ภาคตะวันออก': ['ชลบุรี', 'ระยอง', 'จันทบุรี', 'ตราด', 'ฉะเชิงเทรา'],
        'ภาคตะวันตก': ['กาญจนบุรี', 'ราชบุรี', 'เพชรบุรี', 'ประจวบคีรีขันธ์', 'ตาก'],
        'ภาคใต้': ['สงขลา', 'สุราษฎร์ธานี', 'นครศรีธรรมราช', 'ภูเก็ต', 'กระบี่']
    }
    
    store_types = ['ร้านขนาดใหญ่', 'ร้านขนาดกลาง', 'ร้านขนาดเล็ก']
    
    store_ids = []
    store_names = []
    store_regions = []
    store_provinces = []
    store_types_list = []
    store_opening_dates = []
    
    store_id = 100
    # สร้างวันที่เปิดร้านในช่วง 5 ปีที่ผ่านมา
    end_date = datetime.now()
    start_date = end_date - timedelta(days=5*365)  # 5 ปีย้อนหลัง
    
    for region in regions:
        for province in provinces[region]:
            # สร้างร้านค้า 2-5 ร้านต่อจังหวัด
            num_stores = random.randint(2, 5)
            
            for i in range(num_stores):
                store_id += 1
                store_ids.append(store_id)
                
                store_name = f"ร้าน {province} สาขา {i+1}"
                store_names.append(store_name)
                
                store_regions.append(region)
                store_provinces.append(province)
                
                # สุ่มประเภทร้านค้า โดยเน้นร้านขนาดกลางให้มีมากที่สุด
                weights = [0.2, 0.5, 0.3]  # ใหญ่, กลาง, เล็ก
                store_type = random.choices(store_types, weights=weights)[0]
                store_types_list.append(store_type)
                
                # วันที่เปิดร้านสุ่มในช่วง 5 ปีที่ผ่านมา
                days_delta = (end_date - start_date).days
                random_days = random.randint(0, days_delta)
                opening_date = start_date + timedelta(days=random_days)
                store_opening_dates.append(opening_date.strftime('%Y-%m-%d'))
    
    stores_df = pd.DataFrame({
        'store_id': store_ids,
        'store_name': store_names,
        'region': store_regions,
        'province': store_provinces,
        'store_type': store_types_list,
        'opening_date': store_opening_dates
    })
    
    return stores_df

In [ ]:
# สร้างข้อมูลการขาย
def create_sales_data(products_df, customers_df, stores_df, num_transactions=10000):
    """
    สร้างข้อมูลการขายจากสินค้า ลูกค้า และร้านค้า
    
    Args:
        products_df: DataFrame ข้อมูลสินค้า
        customers_df: DataFrame ข้อมูลลูกค้า
        stores_df: DataFrame ข้อมูลร้านค้า
        num_transactions: จำนวนธุรกรรมการขายที่ต้องการสร้าง
        
    Returns:
        DataFrame ของข้อมูลการขาย
    """
    # สร้างวันที่การขายในช่วง 1 ปีที่ผ่านมา
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365)  # 1 ปีย้อนหลัง
    
    transaction_ids = []
    sale_dates = []
    customer_ids = []
    store_ids = []
    product_ids = []
    quantities = []
    unit_prices = []
    sales_amounts = []
    cost_amounts = []
    payment_methods = ['เงินสด', 'บัตรเครดิต', 'แอปพลิเคชันธนาคาร', 'บัตรเดบิต', 'คิวอาร์โค้ด']
    payment_list = []
    
    transaction_id = 1000000
    days_delta = (end_date - start_date).days
    
    # สุ่มการขายตามจำนวนที่กำหนด
    for _ in range(num_transactions):
        transaction_id += 1
        
        # สุ่มวันที่ขาย
        random_days = random.randint(0, days_delta)
        sale_date = start_date + timedelta(days=random_days)
        
        # สุ่มข้อมูลลูกค้า
        customer = customers_df.sample(1).iloc[0]
        customer_id = customer['customer_id']
        
        # สุ่มร้านค้าในภูมิภาคเดียวกับลูกค้า
        regional_stores = stores_df[stores_df['region'] == customer['region']]
        if not regional_stores.empty:
            store = regional_stores.sample(1).iloc[0]
        else:
            store = stores_df.sample(1).iloc[0]
        store_id = store['store_id']
        
        # จำนวนสินค้าต่อธุรกรรม (1-5 ชิ้น)
        num_items = random.randint(1, 5)
        products_in_transaction = products_df.sample(num_items)
        
        # สร้างข้อมูลการขายสำหรับแต่ละสินค้า
        for _, product in products_in_transaction.iterrows():
            transaction_ids.append(transaction_id)
            sale_dates.append(sale_date.strftime('%Y-%m-%d'))
            customer_ids.append(customer_id)
            store_ids.append(store_id)
            
            product_id = product['product_id']
            product_ids.append(product_id)
            
            # สุ่มจำนวนสินค้า (1-3 ชิ้นต่อรายการ)
            quantity = random.randint(1, 3)
            quantities.append(quantity)
            
            # ราคาและต้นทุนตามข้อมูลสินค้า
            unit_price = product['price']
            unit_prices.append(unit_price)
            
            sales_amount = unit_price * quantity
            sales_amounts.append(sales_amount)
            
            cost_amount = product['cost'] * quantity
            cost_amounts.append(cost_amount)
            
            # สุ่มวิธีการชำระเงิน
            payment = random.choice(payment_methods)
            payment_list.append(payment)
    
    sales_df = pd.DataFrame({
        'transaction_id': transaction_ids,
        'sale_date': sale_dates,
        'customer_id': customer_ids,
        'store_id': store_ids,
        'product_id': product_ids,
        'quantity': quantities,
        'unit_price': unit_prices,
        'sales_amount': sales_amounts,
        'cost_amount': cost_amounts,
        'payment_method': payment_list
    })
    
    return sales_df

In [ ]:
# สร้างข้อมูลทั้งหมด
products_df = create_product_data()
customers_df = create_customer_data()
stores_df = create_store_data()
sales_df = create_sales_data(products_df, customers_df, stores_df, num_transactions=10000)

# แสดงตัวอย่างข้อมูลและข้อมูลสรุป
print("ข้อมูลสินค้า:")
display(products_df.head())
print(f"จำนวนสินค้าทั้งหมด: {len(products_df)}")

print("\nข้อมูลลูกค้า:")
display(customers_df.head())
print(f"จำนวนลูกค้าทั้งหมด: {len(customers_df)}")

print("\nข้อมูลร้านค้า:")
display(stores_df.head())
print(f"จำนวนร้านค้าทั้งหมด: {len(stores_df)}")

print("\nข้อมูลการขาย:")
display(sales_df.head())
print(f"จำนวนรายการขายทั้งหมด: {len(sales_df)}")

### 2.2 การเตรียมข้อมูลเพิ่มเติมสำหรับการวิเคราะห์

เพื่อให้การวิเคราะห์ข้อมูลทำได้ง่ายขึ้น เราจะสร้างคอลัมน์เพิ่มเติมและทำการเชื่อมโยงข้อมูล

In [ ]:
# สร้างคอลัมน์เพิ่มเติมสำหรับการวิเคราะห์
sales_df['profit'] = sales_df['sales_amount'] - sales_df['cost_amount']
sales_df['profit_margin'] = (sales_df['profit'] / sales_df['sales_amount']) * 100
sales_df['sale_date'] = pd.to_datetime(sales_df['sale_date'])
sales_df['month'] = sales_df['sale_date'].dt.strftime('%Y-%m')
sales_df['quarter'] = sales_df['sale_date'].dt.to_period('Q').astype(str)
sales_df['year'] = sales_df['sale_date'].dt.year
sales_df['weekday'] = sales_df['sale_date'].dt.day_name()

# เชื่อมข้อมูลการขายกับข้อมูลอื่นๆ
sales_with_info = sales_df.merge(products_df, on='product_id', how='left')
sales_with_info = sales_with_info.merge(customers_df, on='customer_id', how='left')
sales_with_info = sales_with_info.merge(stores_df, on='store_id', how='left')

print("ข้อมูลการขายที่เชื่อมกับข้อมูลสินค้า ลูกค้า และร้านค้า:")
display(sales_with_info.head())

### 2.3 การส่งออกข้อมูลสำหรับ Power BI

เราจะส่งออกข้อมูลเป็นไฟล์ CSV เพื่อนำเข้า Power BI

In [ ]:
# สร้างโฟลเดอร์สำหรับเก็บไฟล์ CSV (ในกรณีที่ยังไม่มี)
if not os.path.exists('power_bi_data'):
    os.makedirs('power_bi_data')

# ส่งออกข้อมูล
products_df.to_csv('power_bi_data/products.csv', index=False, encoding='utf-8-sig')
customers_df.to_csv('power_bi_data/customers.csv', index=False, encoding='utf-8-sig')
stores_df.to_csv('power_bi_data/stores.csv', index=False, encoding='utf-8-sig')
sales_df.to_csv('power_bi_data/sales.csv', index=False, encoding='utf-8-sig')

print("ส่งออกข้อมูลเรียบร้อยแล้ว! ไฟล์ CSV ถูกบันทึกไว้ในโฟลเดอร์ 'power_bi_data'")